## Paraphrase results
Computes the paraphrasing results.

In [ ]:
# If None, assumes results to be in the "results" directory.
RESULT_DIR = None # e.g. "results_review"

In [ ]:
import pandas as pd
from moralsim.analysis.metrics import compute_metrics_per_scenario, compute_metrics_per_model, get_groups
# Replace models with custom models
models = {
    "GPT-4o": ["z-gpt-4o-2024-08-0", "openai/gpt-4o-2024-08-06"],
    "Deepseek-R1": "deepseek/deepseek-r1",
}
scenarios = {
    "pg_venture_dummy_cooperate_survival_cot": {"group": "pg_venture_dummy_cooperate_survival_cot"},
    "pg_venture_dummy_cooperate_survival_cot_p0": {"group": "pg_venture_dummy_cooperate_survival_cot_p0"},
    "pg_venture_dummy_cooperate_survival_cot_p1": {"group": "pg_venture_dummy_cooperate_survival_cot_p1"},
    "pg_venture_dummy_cooperate_survival_cot_p2": {"group": "pg_venture_dummy_cooperate_survival_cot_p2"},
    "pd_privacy_dummy_defect_cot": {"group": "pd_privacy_dummy_defect_cot"},
    "pd_privacy_dummy_defect_cot_p0": {"group": "pd_privacy_dummy_defect_cot_p0"},
    "pd_privacy_dummy_defect_cot_p1": {"group": "pd_privacy_dummy_defect_cot_p1"},
    "pd_privacy_dummy_defect_cot_p2": {"group": "pd_privacy_dummy_defect_cot_p2"},
}

scenario_metrics, scenario_runs, scenario_metrics_per_run = compute_metrics_per_scenario(models=models, scenarios=scenarios, paraphrase=True, result_dir=RESULT_DIR, save_dir=None)

paraphrases = []
originals = []
for key, val in scenario_metrics.items():
    if key.endswith("cot"):
        val.insert(0, "group", key)
        val.insert(1, "paraphrase", None)
        originals.append(val)
    else:
        val.insert(0, "group", key[:-3])
        val.insert(1, "paraphrase", key[-2:])
        paraphrases.append(val)
paraphrase_df = pd.concat(paraphrases).reset_index(drop=True)
original_df = pd.concat(originals).reset_index(drop=True)


In [3]:
import numpy as np
diffs = {
    "morality": [],
    "payoff": [],
    "opponent": []
}
for id, row in paraphrase_df.iterrows():
    for metric, diff in diffs.items():
        diffs[metric].append(abs(row[metric] - original_df.loc[(original_df["group"] == row["group"]) & (original_df["model"] == row["model"]), metric].item()))
diffs = {k: np.array(val) for k, val in diffs.items()}

In [4]:
for k, v in diffs.items():
    print(f"{k}: \tmean: {v.mean():.3f}\tstd: {v.std():.3f}")

morality: 	mean: 0.018	std: 0.024
payoff: 	mean: 0.021	std: 0.032
opponent: 	mean: 0.018	std: 0.024
